### Import the required packages into python

In [ ]:
import pulp
import itertools
import gurobipy

### Set up Test A problem

In [ ]:
def initialise_testa_minimisation_problem(
    specialties, hospitals, bands, regions, scenarios
):
    """
    Initialise the minimisation problem.
    Set decision variables for the models.
    """
    sh = [(s,h) for s in specialties for h in hospitals]
    shb = [(s,h,b) for s in specialties for h in hospitals for b in bands]
    shk = [(s,h,k) for s in specialties for h in hospitals for k in scenarios]
    srhk = [(s,r,h,k) for s in specialties for r in regions for h in hospitals for k in scenarios]
    sbhk = [(s,b,h,k) for s in specialties for b in bands for h in hospitals for k in scenarios]
    
    prob = pulp.LpProblem("Test A", pulp.LpMinimize)
    
    xstaff = pulp.LpVariable.dicts(
        "Xstaff", (specialties,hospitals,bands), lowBound=0,cat = 'Integer'
    )
    ubed = pulp.LpVariable.dicts(
        "Ubed",(specialties,hospitals,scenarios), lowBound=0, cat='Integer'
    )
    ustaff = pulp.LpVariable.dicts(
        "Ustaff",(specialties,hospitals,bands,scenarios), lowBound=0, cat='Integer'
    )
    return prob, sh, shb, shk, srhk, sbhk, xbed, xstaff, ubed, ustaff
def add_testa_constraints(
    xbed, xstaff, ubed, ustaff, UBbed, UBstaff, UBubed, UBustaff, D, R, K, prob, sh, shb, shk, srhk, sbhk
):
    
    """
    Add the constrains that are required for the Test A model
    
    - Constraints 1-6: Ensures demand is met across all specialties and all regions
    - Constraint 7: Ensures beds are only able to open in a ward if the facilities are able to be opened - 2nd stage
    - Constraint 8: Ensures staffing ratios are met in the first stage
    - Constraint 9: Ensures staffing ratios are met in the second stage
    - Constraint 10 Ensures beds deployed does not exceed maximum capacity of hospital - 2nd stage
    - Constraint 11 Ensures staff deployed does not exceed maximum staffing resources - 1st stage
    - Constraint 12 Ensures staff deployed does not exceed maximum staffing resources - 2nd stage
      """
        
    for k in scenarios:
        for s in specialties:
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region1) >= pulp.lpSum(D[s][0][k]) #Constraint 1
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region2) >= pulp.lpSum(D[s][1][k]) #Constraint 2
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region3) >= pulp.lpSum(D[s][2][k]) #Constraint 3
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region4) >= pulp.lpSum(D[s][3][k]) #Constraint 4
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region5) >= pulp.lpSum(D[s][4][k]) #Constraint 5
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region6) >= pulp.lpSum(D[s][5][k]) #Constraint 6 
                  
   
            
    for s in specialties: 
        for h in hospitals:
            prob += pulp.lpSum(ubed[s][h][k] for k in scenarios)<= pulp.lpSum(K[s][h]) #Constraint 7
            
            for b in bands:
                prob += pulp.lpSum(xstaff[s][h][b]) >= pulp.lpSum(R[s][b]*(xbed[s][h])) #Constraint 8
                
                for k in scenarios:
                    prob += pulp.lpSum(ustaff[s][h][b][k])>= pulp.lpSum(R[s][b]*(ubed[s][h][k])) #Constraint 9
                    
        
    for k in scenarios: 
        for h in hospitals:
            prob += pulp.lpSum(ubed[s][h][k] for s in specialties) <=UBubed[h][k] #Constraint 10
        
    for b in bands:
        prob += pulp.lpSum(xstaff[s][h][b] for (s,h) in sh) <= UBstaff[b] #Constraint 11
        
        for k in scenarios:
            prob += pulp.lpSum(ustaff[s][h][b][k] for (s,h) in sh) <= UBustaff[b][k] #Constraint 12
            
    return prob

def solve_testa_minimisation_problem(
    specialties,
    bands,
    hospitals,
    regions,
    scenarios,
    pscenarios,
    D,
    R,
    K,
    c1bed,
    c2bed,
    c1staff,
    c2staff,
    UBbed,
    UBstaff,
):
    """
    Solves the testa problem, with the objective function being minimised.
    """
    prob, sh, shb, shk, srhk, sbhk, xbed, xstaff, ubed, ustaff = initialise_testa_minimisation_problem(
        specialties=specialties, 
        hospitals=hospitals,
        bands=bands, 
        regions=regions, 
        scenarios=scenarios
    )
    prob +=(
        pulp.lpSum((xbed[s][h]*c1bed[s][h]) for (s,h) in sh) +
        pulp.lpSum((xstaff[s][h][b]*c1staff[b]) for (s,h,b) in shb) + 
        pulp.lpSum(pscenarios[k]*(ubed[s][h][k]*c2bed[s][h]) for (s,h,k) in shk)+ 
        pulp.lpSum(pscenarios[k]*(ustaff[s][h][b][k]*c2staff[b]) for (s,b,h,k) in sbhk)
    )

    prob = add_testa_constraints(
        xbed=xbed, 
        xstaff=xstaff,
        ubed=ubed,
        ustaff=ustaff,
        UBbed=UBbed, 
        UBstaff=UBstaff,
        UBubed=UBubed,
        UBustaff=UBustaff,
        D=D, 
        R=R,
        K=K,
        sh=sh,
        shb=shb,
        shk=shk,
        srhk=srhk,
        sbhk=sbhk,
        prob=prob,
    )
    
    prob.solve(pulp.GUROBI())
    # prob.solve(pulp.PULP_CBC_CMD())
    
    return prob

### The following cell can be edited to allow the user to enter their own parameters and values for the model

In [ ]:
specialties = list(itertools.chain(range(0, 29)))

bands = list(itertools.chain(range(0,2)))
region1 = [0,1,2]
region2 = [3,4]
region3 = [5]
region4 = [6]
region5 = [7,8,9]
region6 = [10]
hospitals = region1 + region2 +region3 + region4 +region5 +region6

regions = list(itertools.chain(range(0,6)))



K = [
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,309,0,0,80],
    [588,0,75,163,0,80,73,309,26,0,0],
    [0,0,0,163,0,0,73,0,26,0,0],
    [588,28,75,163,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,0],
    [588,0,0,163,0,0,0,0,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,80],
    [588,0,0,163,0,80,0,309,0,0,0],
    [588,0,0,163,0,0,0,309,26,0,80],
    [0,0,0,0,12,0,0,0,26,16,80],
    [588,0,0,163,0,0,0,309,0,0,80],
    [588,28,0,163,0,0,0,309,0,0,80],
    [588,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,80,73,0,0,0,0],
    [588,28,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,80],
    [588,0,0,163,0,0,73,309,0,0,0],
    [0,0,0,0,0,0,0,309,0,0,80],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,0,0,0,80],
    [588,28,75,163,0,80,73,309,26,0,0],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,163,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,80]
    ]
   
 
     
R = [
    [0.250,0.250],
    [0.250,0.250],
    [0.125,0.125],
    [0.1,0.1],
    [0.1,0.1],
    [0.1,0.1],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.25,0.25],
    [0.25,0.25],
    [0.1,0.1],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.1,0.1],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.1,0.1],
    [0.125,0.125],
    [0.125,0.125],
    [0.125,0.125],
    [0.25,0.25],
    [0.125,0.125],
    ]
     
c1bed = [
    [345,0,0,0,0,0,0,149,0,0,0],
    [526,0,0,0,0,0,0,1516,0,0,0],
    [396,0,0,0,0,0,0,895,0,0,551],
    [457,0,755,493,0,542,472,743,577,0,0],
    [0,0,0,942,0,0,1100,0,1021,0,0],
    [1672,216,1232,2404,0,0,0,0,0,0,0],
    [296,0,0,1270,0,0,0,1497,0,0,0],
    [481,0,0,501,0,0,0,0,0,0,0],
    [448,402,0,639,0,0,0,959,0,0,832],
    [390,0,0,94,0,65,0,611,0,0,0],
    [472,0,0,304,0,0,0,539,541,0,849],
    [0,0,0,0,172,0,0,0,325,443,360],
    [1007,0,0,528,0,0,0,292,0,0,241],
    [1299,1277,0,1218,0,0,0,1070,0,0,1176],
    [711,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,39,197,0,0,0,0],
    [2026,1940,0,0,0,0,0,264,0,0,0],
    [1273,0,0,0,0,0,0,0,0,0,0],
    [957,0,0,369,0,0,0,1416,0,0,174],
    [124,0,0,111,0,0,134,143,0,0,0],
    [0,0,0,0,0,0,0,1308,0,0,496],
    [945,0,0,0,0,0,0,1097,0,0,0],
    [633,0,0,0,0,0,0,0,0,0,1545],
    [1973,1274,972,975,0,1021,1983,1987,1455,0,0],
    [330,0,0,0,0,0,0,566,0,0,0],
    [141,0,0,139,0,0,0,0,0,0,0],
    [631,0,0,621,0,0,0,536,0,0,0],
    [678,651,0,610,0,0,0,873,0,0,0],
    [102,214,0,79,0,0,0,378,0,0,1122]
    ]

c2bed = [
    [414,0,0,0,0,0,0,178.8,0,0,0],
    [631.2,0,0,0,0,0,0,1819.2,0,0,0],
    [475.2,0,0,0,0,0,0,1074,0,0,661.2],
    [548.4,0,906,591.6,0,650.4,566.4,891.6,692.4,0,0],
    [0,0,0,1130.4,0,0,0,1320,1225.2,0,0],
    [2006.4,259.2,1478.4,2884.8,0,0,0,0,0,0,0],
    [355.2,0,0,1524,0,0,0,1796.4,0,0,0],
    [577.2,0,0,601.2,0,0,0,0,0,0,0],
    [537.6,482.4,0,766.8,0,0,0,1150.8,0,0,998.4],
    [468,0,0,112.8,0,78,0,733.2,0,0,0],
    [566.4,0,0,364.8,0,0,0,646.8,649.2,0,1018.8],
    [0,0,0,0,206.4,0,0,0,390,531.6,432],
    [1208.4,0,0,633.6,0,0,0,350.4,0,0,289.2],
    [1558.8,1532.4,0,1461.6,0,0,0,1284,0,0,1411.2],
    [853.2,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,46.8,236.4,0,0,0,0],
    [2431.2,2328,0,0,0,0,0,316.8,0,0,0],
    [1527.6,0,0,0,0,0,0,0,0,0,0],
    [1148.4,0,0,442.8,0,0,0,1699.2,0,0,208.8],
    [148.8,0,0,133.2,0,0,160.8,171.6,0,0,0],
    [0,0,0,0,0,0,0,1569.6,0,0,595.2],
    [1134,0,0,0,0,0,0,1316.4,0,0,0],
    [759.6,0,0,0,0,0,0,0,0,0,1854],
    [2367.6,1528.8,1166.4,1170,0,1225.2,2379.6,2384.4,1746,0,0],
    [396,0,0,0,0,0,0,679.2,0,0,0],
    [169.2,0,0,166.8,0,0,0,0,0,0,0],
    [757.2,0,0,745.2,0,0,0,643.2,0,0,0],
    [813.6,781.2,0,732,0,0,0,1047.6,0,0,0],
    [122.4,256.8,0,94.8,0,0,0,453.6,0,0,1346.4]
    ]

D = [
    [[2.00269,2.21350,1.89728,2.31890],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[8.72534,9.64379,8.26611,10.10302],[0.00000,0.00000,0.00000,0.00000]],
    [[4.37752,4.83831,4.14713,5.06871],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.73475,0.81209,0.69608,0.85076],[0.00000,0.00000,0.00000,0.00000]],
    [[15.28995,16.89942,14.48522,17.70416],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[9.38685,10.37493,8.89280,10.86898],[0.00018,0.00020,0.00017,0.00021]],
    [[89.81177,99.26564,85.08483,103.99257],[54.85107,60.62486,51.96417,63.51176],[0.71147,0.78636,0.67402,0.82381],[8.30456,9.17873,7.86748,9.61581],[44.15471,48.80257,41.83077,51.12650],[0.00000,0.00000,0.00000,0.00000]],
    [[0.00000,0.00000,0.00000,0.00000],[6.64543,7.34495,6.29567,7.69471],[0.00000,0.00000,0.00000,0.00000],[0.29645,0.32766,0.28085,0.34326],[12.32685,13.62441,11.67807,14.27320],[0.00000,0.00000,0.00000,0.00000]],
    [[2.29823,2.54014,2.17727,2.66110],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[13.83530,15.29165,13.10713,16.01982],[20.21962,22.34800,19.15543,23.41219],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[16.37676,18.10063,15.51483,18.96257],[0.00000,0.00000,0.00000,0.00000]],
    [[3.08560,3.41039,2.92320,3.57279],[0.00391,0.00432,0.00370,0.00452],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[12.36975,13.67182,11.71871,14.32286],[0.57621,0.63687,0.54589,0.66719],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[19.12646,21.13978,18.11981,22.14643],[0.06887,0.07612,0.06524,0.07974]],
    [[80.72259,89.21971,76.47404,93.46827],[0.93539,1.03385,0.88616,1.08308],[0.01091,0.01206,0.01033,0.01263],[0.00000,0.00000,0.00000,0.00000],[13.46105,14.87800,12.75257,15.58648],[0.00000,0.00000,0.00000,0.00000]],
    [[44.25176,48.90984,41.92272,51.23888],[0.51204,0.56594,0.48509,0.59289],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[20.79956,22.98898,19.70484,24.08370],[0.00055,0.00060,0.00052,0.00063]],
    [[0.00000,0.00000,0.00000,0.00000],[9.37969,10.36702,8.88602,10.86069],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[14.52357,16.05237,13.75918,16.81677],[0.00000,0.00000,0.00000,0.00000]],
    [[2.01344,2.22538,1.90747,2.33135],[0.11610,0.12832,0.10999,0.13443],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[1.11301,1.23017,1.05443,1.28875],[0.00018,0.00020,0.00017,0.00021]],
    [[2.91818,3.22536,2.76459,3.37895],[0.03042,0.03363,0.02882,0.03523],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[1.74222,1.92562,1.65053,2.01731],[0.00018,0.00020,0.00017,0.00021]],
    [[6.82266,7.54084,6.46357,7.89992],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.32551,0.35977,0.30838,0.37690],[0.30855,0.34103,0.29231,0.35727],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[1.12390,1.24220,1.06475,1.30136],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.02312,0.02555,0.02190,0.02677],[0.00000,0.00000,0.00000,0.00000]],
    [[1.50370,1.66198,1.42456,1.74112],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[2.42615,2.68154,2.29846,2.80923],[0.00270,0.00298,0.00256,0.00313],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.18701,0.20669,0.17716,0.21653],[0.49704,0.54936,0.47088,0.57552]],
    [[0.05570,0.06156,0.05277,0.06450],[0.00520,0.00575,0.00493,0.00602],[0.00000,0.00000,0.00000,0.00000],[0.00347,0.00384,0.00329,0.00402],[0.01247,0.01379,0.01182,0.01444],[0.00000,0.00000,0.00000,0.00000]],
    [[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.03196,0.03533,0.03028,0.03701],[0.00018,0.00020,0.00017,0.00021]],
    [[0.01385,0.01531,0.01312,0.01604],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00247,0.00273,0.00234,0.00286],[0.00000,0.00000,0.00000,0.00000]],
    [[0.21519,0.23784,0.20386,0.24916],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[59.71795,66.00405,56.57490,69.14710],[31.31702,34.61355,29.66876,36.26181],[65.52250,72.41960,62.07394,75.86815],[30.39211,33.59128,28.79253,35.19087],[23.24258,25.68917,22.01929,26.91246],[0.00000,0.00000,0.00000,0.00000]],
    [[28.31092,31.29102,26.82087,32.78107],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[26.44534,29.22906,25.05348,30.62092],[0.00000,0.00000,0.00000,0.00000]],
    [[0.00008,0.00009,0.00007,0.00009],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000]],
    [[0.00034,0.00038,0.00033,0.00040],[0.00139,0.00154,0.00132,0.00161],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.01219,0.01348,0.01155,0.01412],[0.00000,0.00000,0.00000,0.00000]],
    [[57.29699,63.32825,54.28135,66.34388],[0.63316,0.69981,0.59984,0.73314],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[39.44269,43.59455,37.36676,45.67048],[0.00000,0.00000,0.00000,0.00000]],
    [[11.72267,12.95664,11.10569,13.57362],[0.04951,0.05472,0.04690,0.05732],[0.00000,0.00000,0.00000,0.00000],[0.00000,0.00000,0.00000,0.00000],[0.19324,0.21359,0.18307,0.22376],[0.06667,0.07368,0.06316,0.07719]],
    ]

c1staff = [338.88,419.52]  

c2staff = [454.80,560.64] 

UBbed = [[588], [28], [75], [163], [12], [80], [73], [309], [26], [16], [80]]
        
UBubed = [
    [177,177,177, 177],
    [9,9,9,9],
    [28,28,28,28],
    [49,49,49,49],
    [4,4,4,4],
    [24,24,24,24],
    [22,22,22,22],
    [93,93,93,93],
    [8,8,8,8],
    [5,5,5,5],
    [24,24,24,24]
    ] 

UBstaff = [[400],[400]]

UBustaff = [
    [200,200,200,200],
    [200,200,200,200],
    ]
  
scenarios = [0,1,2,3]

pscenarios = [0.25,0.25,0.25,0.25]   

xbed= [
    [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [18.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 12.,  0.,  0.,  1.],
       [89.,  0.,  0., 59.,  0.,  1.,  0.,  9., 44.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  8.,  0.,  0.,  0.,  1.,  0., 14.,  0.,  0.],
       [ 0.,  3.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [13.,  0.,  0., 22.,  0.,  0.,  0.,  0., 17.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 12.,  0.,  2.,  0.,  0.,  0.,  0., 20.,  0.,  0.,  1.],
       [88.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 17.,  0.,  0.,  0.],
       [41.,  0.,  0.,  1.,  0.,  0.,  0.,  0., 21.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0., 14.,  0.],
       [ 2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 4.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  1.],
       [ 6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 62., 24.,  0., 68.,  0., 33., 13., 12.,  0.,  0.],
       [27.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 29.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [47., 11.,  0.,  1.,  0.,  0.,  0.,  0., 40.,  0.,  0.,  0.],
       [16.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.]
]

### The following runs the Test A model

In [ ]:
prob = solve_testc_minimisation_problem(
    specialties,
    bands,
    hospitals,
    regions,
    scenarios,
    pscenarios,
    D,
    R,
    K,
    c1bed,
    c2bed,
    c1staff,
    c2staff,
    UBbed,
    UBstaff,
    xbed
)

### The following outputs the results of the Test A model

In [ ]:
print("Solution Status = ", pulp.LpStatus[prob.status])
print("Total price = ", pulp.value(prob.objective))  
for v in prob.variables():
    if v.varValue >= 0:
        #print(v.varValue)
        print(v.name, "=", v.varValue)
production = [v.varValue for v in prob.variables()]